In [44]:
import http.client
import json
import pandas as pd

conn = http.client.HTTPSConnection("web.byui.edu")

payload = ""

headers = {'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36"}

conn.request("GET", "/studentemployment/api/jobs", payload, headers)

res = conn.getresponse()

data = res.read()

info = data.decode("utf-8")

responseObject = json.loads(info)

data_jobs = pd.DataFrame(responseObject)

In [51]:
# from datetime import datetime
# data_jobs.to_csv(f'../Job_board - Historical Data/data_jobs {datetime.now().strftime("%d-%m-%Y %H%M%S")}.csv')

pd.set_option('display.max_rows', None)
data_jobs.head()

,title,departmentName,dateUpdated,startDate,endDate,managerName,payRate,beginningDate,recruitingStartDate,limitApplicants,limitNumber
0,Online Teaching Assistant - FHGEN 242,Campus Teaching Assistants,2022-08-17 01:14:09.031840700,2022-08-17 05:00:00,2022-09-12 08:00:00,Jodi Robison,10.0,2022-09-14 02:00:00,2020-10-20 01:00:00,True,50
1,Online Tutor - FHGEN 242,Online Student,2022-08-17 01:13:55.194069400,2022-08-16 06:00:00,2022-09-11 07:00:00,Casey McDaniel,10.0,2022-09-13 07:00:00,2020-11-06 01:00:00,True,50
2,Online Teaching Assistant - FCS 360,Campus Teaching Assistants,2022-08-17 01:11:52.619672600,2022-08-17 04:00:00,2022-10-02 02:00:00,Jodi Robison,10.0,2022-09-15 02:00:00,2022-03-08 01:00:00,False,0
3,Online Teaching Assistant - FCS 360,Online Student,2022-08-17 01:10:57.195746500,2022-08-17 04:00:00,2022-10-03 02:00:00,Jodi Robison,10.0,2022-09-15 02:00:00,2022-03-08 01:00:00,False,0
4,Online Teaching Assistant - AGBUS 138,Campus Teaching Assistants,2022-08-16 21:43:56.571029200,2022-06-27 02:00:00,2022-09-13 04:00:00,Jodi Robison,10.0,2022-09-13 04:00:00,2020-10-20 01:00:00,False,0


In [47]:
data_jobs.dateUpdated = data_jobs.dateUpdated.apply(lambda x: pd.to_datetime(x))

data_jobs.startDate = data_jobs.startDate.apply(lambda x: pd.to_datetime(x))

data_jobs.endDate = data_jobs.endDate.apply(lambda x: pd.to_datetime(x))

data_jobs.beginningDate = data_jobs.beginningDate.apply(lambda x: pd.to_datetime(x))

data_jobs.recruitingStartDate = data_jobs.recruitingStartDate.apply(lambda x: pd.to_datetime(x))

columns_to_drop = [
    'jobID', # not needed for EDA
    'description', # not needed for EDA
    'summary', # not needed for EDA
    'displayJob', # single boolean
    'approximateHoursPerWeek', # not consistent
    'positionsAllocated', # not relevant
    'positionsAvailble', # not relevant
    'workSchedule', # not needed for EDA
    'requireResume', # not relevant
    'applicants', # empty
    'jobQuestions', # empty
    'isOnline', # not accurate
    'allowOnline', # not accurate
    'jobMajors' # not relevant
    ]
data_jobs.drop(columns_to_drop, axis=1, inplace=True)


In [48]:
data_jobs.columns

Index(['title', 'departmentName', 'dateUpdated', 'startDate', 'endDate',
       'managerName', 'payRate', 'beginningDate', 'recruitingStartDate',
       'limitApplicants', 'limitNumber'],
      dtype='object')

In [49]:
remove = ['Online','Custodian','TA']
data_jobs[~data_jobs["title"].str.contains('|'.join(remove))].sort_values('payRate',ascending=False)

,title,departmentName,dateUpdated,startDate,endDate,managerName,payRate,beginningDate,recruitingStartDate,limitApplicants,limitNumber
64,Catering Services Lead,University Food Services,2022-08-15 19:22:16.682749200,2022-08-15 12:00:00,2022-09-05 18:00:00,Jesus Del Barrio Sanchez,12.00,2022-09-12 18:00:00,2022-07-28 01:00:00,False,0
307,Health Center Lab Assistant,Student Health Center,2022-08-04 16:32:49.019190400,2022-07-19 12:00:00,2022-08-31 12:00:00,Emma Taylor,11.50,2022-09-12 12:00:00,2022-04-20 01:00:00,False,0
117,Clinician Aide,Student Health Center,2022-08-12 13:58:29.809263500,2022-08-12 18:00:00,2022-08-31 18:00:00,Emma Taylor,10.15,2022-09-01 12:00:00,2022-03-21 01:00:00,False,0
333,Graphic Designer,Music,2022-07-26 21:59:11.565588300,2022-07-27 02:00:00,2022-09-02 06:00:00,Maria Nate,10.00,2022-09-02 06:00:00,2018-07-23 01:00:00,False,0
65,Food Services Student Jobs,University Food Services,2022-08-15 18:47:15.021023100,2022-08-13 12:00:00,2022-09-14 18:00:00,Fauneil Schultz,10.00,2022-09-14 18:00:00,2022-06-01 01:00:00,False,0
134,Career Success Mentor,Career Center,2022-08-10 20:16:20.952075500,2022-08-10 14:00:00,2022-08-24 13:00:00,Sheila Wener,10.00,2022-08-29 14:00:00,2019-11-14 01:00:00,False,0
334,Employer Relations Content Creator,Career & Employment Services,2022-07-26 21:15:55.981764000,2022-08-15 06:00:00,2022-08-29 06:00:00,Janet Barton,10.00,2022-09-05 06:00:00,2022-07-26 01:00:00,False,0
323,Student Project Manager,Online Curriculum Development,2022-07-29 15:12:11.449001200,2022-07-29 12:00:00,2022-09-09 12:00:00,Brian Carter,10.00,2022-09-09 12:00:00,2016-11-28 01:00:00,True,30
160,Ropes Course Facilitator,Student Activities,2022-08-09 19:42:12.258905000,2022-08-02 09:00:00,2022-08-31 22:00:00,Jason Thornton,9.60,2022-09-06 06:00:00,2016-09-01 01:00:00,False,0
118,MC Event Set-Up,MC Day FMS,2022-08-12 01:42:31.946157100,2022-04-30 00:00:00,NaT,Danyelle Johnson,9.58,NaT,2021-10-28 01:00:00,False,0


In [36]:
a = '' OR ''#'


SyntaxError: invalid syntax (<ipython-input-36-7db3d3da6486>, line 1)

In [35]:
a.find('o')

2